## Check the Gpu

In [ ]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


## Get Helper Function

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2025-05-10 08:52:03--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2025-05-10 08:52:03 (9.58 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# import series of helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys


## Download a Text Datasets

In [ ]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# unzip the datasets using helper function
unzip_data("nlp_getting_started.zip")

--2025-05-10 08:52:04--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.174.207, 74.125.23.207, 74.125.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.174.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   957KB/s    in 0.6s    

2025-05-10 08:52:05 (957 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing the text data

In [ ]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")


In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df["text"][2]

"All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"

In [ ]:
# Shuffle the training dataframe
train_df_shuffled=train_df.sample(frac=1,random_state=23)
train_df_shuffled.head()

,id,keyword,location,text,target
1461,2109,catastrophe,NaN,.@robdelaney Catastrophe is anything but! I l...,1
1837,2642,crashed,too far,He was only .4 of a second faster than me and ...,0
6144,8763,siren,NaN,Internet firms to be subject to new cybersecur...,0
2354,3389,demolition,"ÌÏT: 36.142163,-95.979189",@samajp32 really needs to tone it down some in...,0
4765,6781,lightning,Elchilicitanierraversal,#NowPlaying 'The Lightning Strike' de Snow Pat...,0


In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# How many example of each class
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [ ]:
# How many sample total
print(f"total training sample :{len(train_df)}")
print(f"total test sample :{len(test_df)}")
print(f"total samples in data sets : {len(train_df) + len(test_df)}")

total training sample :7613
total test sample :3263
total samples in data sets : 10876


In [ ]:
# lets visualize some random training examples
import random
random_index =random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index : random_index +2].itertuples():
  _,text,target = row
  print(f"Target :{target}","(real disaster)" if target >0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target :1 (real disaster)
Text:
Some great footage of STRONG work from San Bernardino County Fire who is also working the wild fires right now.... http://t.co/QCYQHvn2Ha

---

Target :0 (not real disaster)
Text:
*screams in 25 different languages*

---



## Split data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

train_sentences,val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                         train_df_shuffled["target"].to_numpy(),
                                                                         test_size=0.1,
                                                                         random_state=23)

In [ ]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [ ]:
train_sentences[:10],train_labels[:10]

(array(['The Catastrophic Effects of Hiroshima and Nagasaki Atomic Bombings Still Being Felt Today http://t.co/QVlxpyyyCd',
        'thinking of the time that my friend bailed the nite b4 a dead show...went alone &amp; had a GREAT time. All alone and free to dance. Front row',
        'New technology designed to help prevent dangerous police chases: In an effort to reduce injuries Milwaukee police\x89Û_ http://t.co/cedjdlPDAN',
        'The Great Upheaval By Winik Jay http://t.co/Ef4swP9SXZ http://t.co/Nb7MAAAOfs',
        'Governor allows parole for California school bus hijacker who kidnapped 26 children in 1976. http://t.co/hdAhLgrprl http://t.co/Z1s3T77P3L',
        'Picking up flattened hay men (@ Masonite in Wahpeton ND) https://t.co/Kw3vq4niJQ',
        'Worked at a fast food joint. Poured burnt hot oil down the sink. It hit the water in the trap and exploded.  @FallonTonight #WORSTSUMMERJOB',
        'Obama declares disaster for typhoon-devastated Saipan #Worldnews http://t.co/

## Converting Text into Numbers

In [ ]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorized=TextVectorization(max_tokens=None,
                                  standardize="lower_and_strip_punctuation",
                                  split="whitespace",
                                  ngrams=None,
                                  output_mode="int",
                                  output_sequence_length=None)

In [ ]:
# find average no. of tokens in training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
# setup text vectorization with custom variable
max_vocab_length=10000
max_length=15

text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                 output_mode="int",
                                 output_sequence_length=max_length)

In [ ]:
# FIT The text vectorizer to training set
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create sample sentence and tokanize it
sample_sentence="You will get what you deserve no worry"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  12,   39,   49,   53,   12, 5773,   42, 1166,    0,    0,    0,
           0,    0,    0,    0]])>

In [ ]:
# choose the random sentence and tokenize it

random_sentence=random.choice(train_sentences)
print(f"Original sentence :\n {random_sentence}\n\
       nVectorized version:")
text_vectorizer([random_sentence])

Original sentence :
 Russian customs destroyed a total of 319 tons of food today phew! Some Italian meats were burned in an incinerator in Pulkovo airport.
       nVectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 803, 5817,  352,    3, 1346,    6,    1, 3303,    6,  259,  126,
        4874,   81, 2061,    1]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")


Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Bottom 5 least common words: [np.str_('overturns'), np.str_('overturned'), np.str_('overturn'), np.str_('overtook'), np.str_('overtime')]


## Creating an Embedding using an Embedding Layer

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

embedding

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding_1, built=False>

## Modelling a text data sets

1. model 0: Navie Bayes (skleran)
2. model 1: Feed Forward neural network(dense model)
3. model 2: LSTM Model(RNN)
4. model 3: GRU Model(RNN)
5. model 4: Bidirectional-LSTM model(RNN)
6. model 5: 1D Convulutional neural network(CNN)
7. model 6: Tensorflow Hub Pretrained Feature Extractor.
8. model 7: Same as model 6 with 10% of training data

## Model 0: Using sklearn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


# Create Tokanization and pipline modelling
model_0= Pipeline([
                  ("tfidf", TfidfVectorizer()),
                  ("clf",MultinomialNB())
])

# fit the model
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# FInd the score on validation data set

baseline_score= model_0.score(val_sentences,val_labels)
print(f"Our Baseline_model accuracy is : {baseline_score *100:.2f}%")

Our Baseline_model accuracy is : 80.45%


In [ ]:
# Make Predictions
baseline_preds=model_0.predict(val_sentences)
baseline_preds[:20]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0])

## Creating a Evaluation Function for our Model Experiments

* Accuracy
* Precision
* Recall
* F1_score

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def Calculate_results(y_true,y_preds):

  # Calculate Model Accuracy
  model_accuracy= accuracy_score(y_true,y_preds)*100
  # Calculate recision,recall,f1score using Weighted average
  model_precision,recall_score,f1_score,_= precision_recall_fscore_support(y_true,y_preds,average="weighted")
  model_results={"accuracy": model_accuracy,
                 "precision": model_precision,
                 "recall": recall_score,
                 "f1_score": f1_score}

  return model_results

In [ ]:
model_0_results=Calculate_results(y_true=val_labels,y_preds=baseline_preds)
model_0_results

{'accuracy': 80.4461942257218,
 'precision': 0.8171285840896145,
 'recall': 0.8044619422572179,
 'f1_score': 0.7972256368332269}

## Model_1 : Dense Model


In [ ]:
# Create a tensorboard_callback(need to create new one for every model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR ="model_log"

In [ ]:
# Build Model with Functional Api
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x= embedding(x)
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_dense")

In [ ]:
# compile a model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit the model
history_model_1=model_1.fit(train_sentences,
                            train_labels,
                            epochs=7,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_1_Dense")])

Saving TensorBoard log files to: model_log/model_1_Dense/20250510-085208
Epoch 1/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.6369 - loss: 0.6488 - val_accuracy: 0.7703 - val_loss: 0.5287
Epoch 2/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.8149 - loss: 0.4677 - val_accuracy: 0.7953 - val_loss: 0.4668
Epoch 3/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8544 - loss: 0.3622 - val_accuracy: 0.7913 - val_loss: 0.4567
Epoch 4/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8862 - loss: 0.2946 - val_accuracy: 0.7861 - val_loss: 0.4690
Epoch 5/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9064 - loss: 0.2450 - val_accuracy: 0.7795 - val_loss: 0.4929
Epoch 6/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9227 - loss: 0.2067 - val_accuracy: 0.7782 - val_loss: 0.5229
Epoch 7/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9407 - loss: 0.1765 - val_accuracy: 0.7703 - val_loss: 0.5565


In [ ]:
# Evaluate on validation data
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7761 - loss: 0.5748


[0.5565392374992371, 0.7703412175178528]

In [ ]:
model_1_preds_probs=model_1.predict(val_sentences)
model_1_preds_probs[:20]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


array([[0.2244699 ],
       [0.03297953],
       [0.04595242],
       [0.3669543 ],
       [0.11982198],
       [0.0089159 ],
       [0.13301694],
       [0.99798924],
       [0.387671  ],
       [0.36327282],
       [0.786122  ],
       [0.94461614],
       [0.67386365],
       [0.3146651 ],
       [0.4457931 ],
       [0.09454288],
       [0.9175276 ],
       [0.9995621 ],
       [0.99645936],
       [0.9495603 ]], dtype=float32)

In [ ]:
# to get preds we need to round and squeeze
model_1_preds=tf.squeeze(tf.round(model_1_preds_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
       1., 1., 1.], dtype=float32)>

In [ ]:
# Calculating all the metrics score
model_1_results=Calculate_results(y_true=val_labels,y_preds=model_1_preds)
model_1_results

{'accuracy': 77.03412073490814,
 'precision': 0.7708734874750099,
 'recall': 0.7703412073490814,
 'f1_score': 0.7705739344974044}

In [ ]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

## Building RNN Models

## Buiding RNN : Model_2_LSTM

In [ ]:
# BUILD LSTM Model
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
print(x.shape)
x=layers.LSTM(64,return_sequences=True)(x)
print(x.shape)
x=layers.LSTM(64)(x)
print(x.shape)
x=layers.Dense(64,activation="relu")(x)
print(x.shape)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_2_LSTM=tf.keras.Model(inputs,outputs,name="model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)
(None, 64)


In [ ]:
#Compile a model
model_2_LSTM.compile(loss="binary_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
# Fit the model
model_2_LSTM.fit(train_sentences,
                 train_labels,
                 epochs=7,
                 validation_data=(val_sentences,val_labels),
                 callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: model_log/model_2_LSTM/20250510-085840
Epoch 1/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 65ms/step - accuracy: 0.9269 - loss: 0.2612 - val_accuracy: 0.7651 - val_loss: 0.6962
Epoch 2/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 14s 64ms/step - accuracy: 0.9505 - loss: 0.1346 - val_accuracy: 0.7690 - val_loss: 0.8284
Epoch 3/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - accuracy: 0.9595 - loss: 0.1091 - val_accuracy: 0.7572 - val_loss: 0.9740
Epoch 4/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 14s 63ms/step - accuracy: 0.9676 - loss: 0.0902 - val_accuracy: 0.7533 - val_loss: 1.0736
Epoch 5/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 19s 56ms/step - accuracy: 0.9702 - loss: 0.0740 - val_accuracy: 0.7677 - val_loss: 1.0739
Epoch 6/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.9719 - loss: 0.0645 - val_accuracy: 0.7703 - val_loss: 1.2740
Epoch 7/7
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.9716 - loss: 0.0654 - val_accuracy: 0.7651 - val_loss: 1.1542


In [ ]:
# model predict
model_2_LSTM_pred_probs=model_2_LSTM.predict(val_sentences)
model_2_LSTM_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step


array([[2.49467394e-03],
       [5.36573876e-04],
       [3.68286669e-03],
       [1.02133214e-01],
       [1.67458188e-02],
       [4.67650680e-04],
       [5.98963834e-02],
       [9.99996662e-01],
       [7.18367934e-01],
       [3.61410677e-02]], dtype=float32)

In [ ]:
# Converting the prediction in the form of val_labels format
model_2_preds=tf.squeeze(tf.round(model_2_LSTM_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 1., 1., 0.], dtype=float32)>

In [ ]:
# Calculating all the results using function
model_2_results=Calculate_results(y_true=val_labels,y_preds=model_2_preds)
model_2_results

{'accuracy': 76.50918635170603,
 'precision': 0.7656342998986443,
 'recall': 0.7650918635170604,
 'f1_score': 0.7653299101430593}

In [ ]:
Calculate_results(y_true=val_labels,y_preds=baseline_preds)

{'accuracy': 80.4461942257218,
 'precision': 0.8171285840896145,
 'recall': 0.8044619422572179,
 'f1_score': 0.7972256368332269}

In [ ]:
model_2_LSTM.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 15, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,099,973 (15.64 MB)

 Trainable params: 1,366,657 (5.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,733,316 (10.43 MB)

## Model_3: RNN component GRU

In [ ]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
#Compile a model
model_3.compile(loss="binary_crossentropy",
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=["accuracy"])

In [ ]:
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_11 (GRU)                    │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_log/GRU/20250510-101055
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.6389 - loss: 0.6142 - val_accuracy: 0.7861 - val_loss: 0.4652
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.8524 - loss: 0.3506 - val_accuracy: 0.7743 - val_loss: 0.4949
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.9099 - loss: 0.2388 - val_accuracy: 0.7310 - val_loss: 0.6223
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.9435 - loss: 0.1692 - val_accuracy: 0.7192 - val_loss: 0.7908
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.9579 - loss: 0.1303 - val_accuracy: 0.7362 - val_loss: 0.7330


In [ ]:
# model_pred probs
model_3_pred_probs=model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


array([[0.12813339],
       [0.0110204 ],
       [0.02060376],
       [0.33948827],
       [0.05338852],
       [0.01642163],
       [0.04427181],
       [0.99533343],
       [0.5295754 ],
       [0.14946108]], dtype=float32)

In [ ]:
# converting the preds to compare it with true labels
model_3_preds=tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 1., 1., 0.], dtype=float32)>

In [ ]:
# Calculating all the results
model_3_results=Calculate_results(y_true=val_labels,y_preds=model_3_preds)
model_3_results

{'accuracy': 73.62204724409449,
 'precision': 0.7411930181399164,
 'recall': 0.7362204724409449,
 'f1_score': 0.7374911874576902}